In [ ]:
!pip install openai
!pip install pandas
!pip install sentence-transformers
!pip install transformers
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
from openai import OpenAI
client = OpenAI(api_key='API')
def gpt(a):
  prompt=a
  response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
      {
        "role": "user",
        "content": prompt

      }
    ],
    temperature=1,
    max_tokens=1000,
    top_p=0.9,
    frequency_penalty=0,
    presence_penalty=0
    )
  generated_text = response.choices[0].message
  #print(generated_text.content)
  return generated_text.content

### definition of similarity function


In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_sim = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2', device=device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
#passage='What is the efficiency of cellular respiration if a cell generates 32 ATP molecule per molecule of glucose? The efficiency of cellular respiration if a cell generates 32 ATP x 7 kcal / 686... What is total number of atp molecules produced in cellular respiration from one glucose There are a total number of 38 molecules of ATP produced in cellular respiration... What is the product of Cellular respiration uses 1 molecule of glucose to produce how many ATP molecules? A: 2 ATP molecules are produced as energy. Anymore questions? In cellular respiration, ATP hydrolysis plays a critical role in providing energy for cellular work.'
def get_sim(passage1,passage2):
  embeddings1 = model_sim.encode(passage1, convert_to_tensor=True)
  embeddings2 = model_sim.encode(passage2, convert_to_tensor=True)
  cosine_similarity = util.pytorch_cos_sim(embeddings1, embeddings2)
  return cosine_similarity.item()

In [ ]:
import torch
import numpy as np
from transformers import BertTokenizerFast, AutoModelForSequenceClassification
tokenizer = BertTokenizerFast.from_pretrained("nboost/pt-bert-base-uncased-msmarco")
model = AutoModelForSequenceClassification.from_pretrained("nboost/pt-bert-base-uncased-msmarco")
model.to(device)
model.eval()
for param in model.parameters():
    param.requires_grad = False
max_length = 512
def get_scores_nbbert(query_list,passage):
    query_passage_pairs = [(query, passage) for query in query_list]
    scores = []
    batch_size = 2
    for i in range(0, len(query_passage_pairs), batch_size):
        batch_pairs = query_passage_pairs[i:i + batch_size]
        batch_encoding = tokenizer(batch_pairs,
                                max_length=max_length,
                                padding='max_length',
                                truncation=True,
                                return_tensors='pt')

        input_ids = batch_encoding['input_ids'].to(device)
        token_type_ids = batch_encoding['token_type_ids'].to(device)
        attention_mask = batch_encoding['attention_mask'].to(device)

        with torch.no_grad():
            outputs = model(input_ids=input_ids,
                            attention_mask=attention_mask,
                            token_type_ids=token_type_ids)
        logits = outputs.logits
        batch_scores = logits[:, 1].cpu().numpy().tolist()
        #print(batch_scores)
        if isinstance(batch_scores, float):
            batch_scores = [batch_scores]
        scores.extend(batch_scores)
    average_score = np.mean(scores)
    print(f"Average Score: {average_score}")
    return average_score


tokenizer_config.json:   0%|          | 0.00/58.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/408 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

### know_attack pipeline

In [ ]:
def step_1_phrase(topk, query_list):
    prompt_1 = f'''Task:
Extract the top {topk} main keywords, key phrases, or concepts from the following query_list. Ensure that these keywords contain complete information and are highly relevant across all queries. Focus on keywords that can significantly improve retrieval relevance.

query_list:
{query_list}

Requirements:
- Extract keywords or key phrases that encapsulate complete information.
- Select keywords that are important and common to as many queries as possible.
- Emphasize keywords that are highly relevant to retrieval and will enhance search accuracy.

Output Format:
- A list of keywords ordered by importance.
'''

    keywords = gpt(prompt_1)
    return keywords


def step_2(keywords, target_passage):
    # Step 2: Analyze how well the passage matches keywords
    prompt_step2 = f'''Task:
You are to analyze how well the following passage matches the given list of keywords.

Steps:
1. **Read the passage carefully.**
2. **Go through each keyword in the Keywords List and check if it appears in the passage.**
3. **Create two lists:**
   - Included Keywords: Keywords that are present in the passage.
   - Missing or Under-emphasized Keywords: Keywords that are not present or not sufficiently covered.
4. **For the Missing or Under-emphasized Keywords, provide a brief note on their importance and suggest how they could be incorporated into the passage to improve it.**
5. **Write a brief analysis summarizing how well the passage covers the keywords and where it can be improved.**

Original Passage:
{target_passage}

Keywords List:
{keywords}

Output Format:
- **Included Keywords:**
  - [Keyword 1]
  - [Keyword 2]
- **Missing or Under-emphasized Keywords:**
  - [Keyword 3] (Missing/Under-emphasized) - [Brief note]
- **Analysis:**
  - [Summary of how the passage matches or lacks certain keywords]
'''
    analysis = gpt(prompt_step2)
    return analysis

In [ ]:
def step_3_augment(analysis, target_passage, num, label):
    # Step 3: Generate the most important adjustments
    # label: 0 for negative stance, 1 for positive stance
    stance = "negative" if label == 0 else "positive"

    prompt_step3 = f'''You are an AI assistant tasked with suggesting adjustments to a passage based on an analysis.

**Analysis:**
{analysis}

**Original Passage:**
{target_passage}

**Instructions:**
- Identify the **{num} most critical adjustments** to include missing keywords or strengthen certain concepts.
- **Important Constraints:**
  - The type of **"Add a short sentence" can only be used ONCE** in all the adjustments.
  - All adjustments must **maintain and reinforce a {stance} stance** in the passage.
  - Do not introduce any content that contradicts or alters the {stance} stance.
- For each adjustment, provide:
  - **Keyword:** The missing or under-emphasized keyword.
  - **Type of Adjustment:** Choose one of the following options:
    - Add a short sentence (maximum 10 tokens) **(can only be used once)**
    - Replace word with a synonym
    - Rewrite sentence without changing the original meaning
  - **Location:** Specify where the adjustment should be made (e.g., after a specific sentence, or indicate the original text to be replaced).
  - **Original Text:** Include the exact text from the passage that needs to be changed (if applicable).
  - **Suggested Change:** Provide the new text including the keyword, ensuring it aligns with the {stance} stance.
- Ensure the adjustments are concise, within the token limits, and **align with the {stance} stance**.

**Output Format:**

**Suggested Adjustments:**

**Adjustment 1:**
- **Keyword:** [Keyword]
- **Type of Adjustment:** [Type]
- **Location:** [Location]
- **Original Text:** "[Original Text]" *(if applicable)*
- **Suggested Change:** "[Suggested Change]"

(Repeat for Adjustments 2 to {num})

**Remember:**
- You must only provide the **{num} most important** adjustments.
- The type of **"Add a short sentence" can only be used ONCE** in all the adjustments.
- The total token difference between the modified passage and the original must be within **15 tokens**.
- **All adjustments must reinforce the {stance} stance and not change the original stance.**

'''
    adjustment = gpt(prompt_step3)
    return adjustment

def step_3_restrict(analysis, target_passage, num, label):
    # Step 3: Generate the most important adjustments
    # label: 0 for negative stance, 1 for positive stance
    stance = "negative" if label == 0 else "positive"

    prompt_step3 = f'''You are an AI assistant tasked with suggesting adjustments to a passage based on an analysis.

**Analysis:**
{analysis}

**Original Passage:**
{target_passage}

**Instructions:**
- Identify the **{num} most critical adjustments** to include missing keywords or strengthen certain concepts.
- **Important Constraints:**
  - All adjustments must **maintain and reinforce a {stance} stance** in the passage.
  - Do not introduce any content that contradicts or alters the {stance} stance.
- For each adjustment, provide:
  - **Keyword:** The missing or under-emphasized keyword.
  - **Type of Adjustment:** Choose one of the following options:
    - Replace word with a synonym
    - Rewrite sentence without changing the original meaning
  - **Location:** Specify where the adjustment should be made (e.g., after a specific sentence, or indicate the original text to be replaced).
  - **Original Text:** Include the exact text from the passage that needs to be changed (if applicable).
  - **Suggested Change:** Provide the new text including the keyword, ensuring it aligns with the {stance} stance.
- Ensure the adjustments are concise, within the token limits, and **align with the {stance} stance**.

**Output Format:**

**Suggested Adjustments:**

**Adjustment 1:**
- **Keyword:** [Keyword]
- **Type of Adjustment:** [Type]
- **Location:** [Location]
- **Original Text:** "[Original Text]" *(if applicable)*
- **Suggested Change:** "[Suggested Change]"

(Repeat for Adjustments 2 to {num})

**Remember:**
- You must only provide the **{num} most important** adjustments.
- The total token difference between the modified passage and the original must be within **10 tokens**.
- **All adjustments must reinforce the {stance} stance and not change the original stance.**

'''
    adjustment = gpt(prompt_step3)
    return adjustment

def step_4(adjustment, target_passage, label):
    # Step 4: Apply the adjustments to the original passage
    # label: 0 for negative stance, 1 for positive stance
    stance = "negative" if label == 0 else "positive"

    prompt_step4 = f'''You are an AI assistant tasked with applying specific adjustments to a passage.

**Original Passage:**
{target_passage}

**Adjustment Suggestions:**
{adjustment}

**Instructions:**
- **Important Constraints:**
  - Only apply the "Add a short sentence" adjustment **ONCE**. If there are multiple such adjustments, only apply the first one.
  - All adjustments must **maintain and reinforce a {stance} stance** in the passage.
  - Do not introduce any content that contradicts or alters the {stance} stance.
- Apply each adjustment **exactly as specified**.
- Locate the exact place in the text for each adjustment using the provided **Original Text** or **Location**.
- **Do not make any other changes** to the other content in the original passage.
- Retain all parts of the original passage that are not affected by the adjustments.
- Ensure the modified passage is coherent, reads naturally, and **reflects the {stance} stance**.

**Output Format:**
[Modified Passage]

Please return the modified passage directly without any other content.and don't output in the format of "[Modified Passage]/n...",directly output the modified passage

'''
    modified_passage = gpt(prompt_step4)
    return modified_passage


In [ ]:
def get_tokens_num(input_string):
  tokens = tokenizer.encode(input_string, add_special_tokens=True)
  num_tokens = len(tokens)
  return num_tokens

def cac_num_pre(target_passage,modified_passage):
    original_tokens = len(target_passage.split())
    modified_tokens = len(modified_passage.split())
    token_difference = abs(modified_tokens - original_tokens)
    token_difference_percentage = token_difference / original_tokens
    return token_difference_percentage


def filter_passage(passage_list, target_passage,rel_limit=0.85, tokens_limit=0.3):
    filter_list = []
    tokens_num_ori = get_tokens_num(target_passage)
    for item in passage_list:
        precentage = cac_num_pre(target_passage, item)
        rel_scores = get_sim(item, target_passage)
        if  precentage <= tokens_limit and rel_scores>=rel_limit:
            filter_list.append(item)
    if not filter_list:
        return None

    return filter_list

def find_best_passage(filter_list, target_passage, query_list):
    if not filter_list:
        return None

    scores_list = []
    for item in filter_list:
        score = get_scores_nbbert(query_list, item)
        scores_list.append((item, score))  # 保存item和其对应的分数

    best_item = max(scores_list, key=lambda x: x[1])  # 根据分数选择最高的item
    return best_item[0]


In [ ]:
def know_attack_augment(query_list,target_passage,label=0,topk=5,num=3):
  keywords=step_1_phrase(topk,query_list)
  analysis=step_2(keywords,target_passage)
  adjustment=step_3_augment(analysis,target_passage,num,label)
  passage1=step_4(adjustment,target_passage,label)
  output_data = {
        "keywords": keywords,
        "analysis": analysis,
        "adjustment": adjustment,
        "passage": passage1,

    }
  return output_data

def know_attack_restrict(query_list,target_passage,label,topk=5,num=3):
  keywords=step_1_phrase(topk,query_list)
  analysis=step_2(keywords,target_passage)
  adjustment=step_3_restrict(analysis,target_passage,num,label)
  passage1=step_4(adjustment,target_passage,label)
  #passage2=adjust_passage_length(target_passage,passage1,adjustment)
  output_data = {
        "keywords": keywords,
        "analysis": analysis,
        "adjustment": adjustment,
        "passage": passage1,

    }
  return output_data


In [ ]:
def sample_passage(query_list,target_passage,label=0,iter=5,topk=10,num=2,augment=True):
  passage_list=[]
  all_data=[]
  tokens_num_ori=get_tokens_num(target_passage)
  for i in range(iter):
    if augment is True:
      output=know_attack_augment(query_list,target_passage,label,topk,num)
    else:
      output=know_attack_restrict(query_list,target_passage,label,topk,num)
    modified_pas=output['passage']
    dif_tokens=cac_num_pre(target_passage,modified_pas)
    all_data.append(output)
    print(dif_tokens)
    print(get_sim(modified_pas,target_passage))
    passage_list.append(modified_pas)
  if not passage_list:
    return [target_passage], all_data
  return passage_list,all_data

def final_attack(query_list,target_passage,label,iter=3,ac_expect=5,topk=10,num=3,):
  ac=0
  print(target_passage)
  t=num
  pas_list=[]
  data_process=[]
  while ac<ac_expect:
    if t==0:
      augment=False
      t+=1
      print(t)
      generate_list,data_1 = sample_passage(query_list, target_passage,label, iter, topk,t,augment)
    else:
      print(t)
      generate_list,data_1 = sample_passage(query_list, target_passage,label, iter, topk,t)
    data_process.extend(data_1)
    filter_list = filter_passage(generate_list, target_passage)
    if filter_list is None:
      t-=1
      ac+=1
    else:
      print(filter_list)
      curr_best=find_best_passage(filter_list,target_passage,query_list)
      dif_token=cac_num_pre(target_passage,curr_best)
      pas_list.extend(filter_list)
      t+=1
      if dif_token>=0.15:
        t-=1
      ac+=1
  if not pas_list:
    return [target_passage], data_process

  return pas_list,data_process


def save_to_json(pas, data_process,num):
    with open(f'{num}data_process.json', 'w') as dp_file:
        json.dump(data_process, dp_file, indent=4)
    with open(f'{num}best_pas.json', 'w') as pas_file:
        json.dump({"best_passage": pas}, pas_file, indent=4)

### Doc_know_generation

In [ ]:
from transformers import pipeline
model_path='cardiffnlp/twitter-roberta-base-sentiment-latest'
device='cuda'
sentiment_task = pipeline("sentiment-analysis", model=model_path, tokenizer=model_path,device=device)

def get_top_positive_passages(passage_list, top_k=5):
    passages_over_40_tokens = []
    for p, n in passage_list:
        if get_tokens_num(p) > 40:
            passages_over_40_tokens.append((p, n))

    if not passages_over_40_tokens:
        return []

    passages = [p for p, n in passages_over_40_tokens]
    nums = [n for p, n in passages_over_40_tokens]
    results = sentiment_task(passages)
    combined_list = []
    for (p, n), r in zip(passages_over_40_tokens, results):
        if r['label'].lower() == 'neutral':
            combined_list.append((p, n, r['score']))
    sorted_combined = sorted(combined_list, key=lambda x: x[2], reverse=True)
    negative_passages = [(p, n) for p, n, score in sorted_combined]
    return negative_passages[:top_k]



def get_top_negative_passages(passage_list, top_k=5):
    passages_over_40_tokens = []
    for p, n in passage_list:
        if get_tokens_num(p) > 40:
            passages_over_40_tokens.append((p, n))

    if not passages_over_40_tokens:
        return []

    passages = [p for p, n in passages_over_40_tokens]
    nums = [n for p, n in passages_over_40_tokens]
    results = sentiment_task(passages)
    combined_list = []
    for (p, n), r in zip(passages_over_40_tokens, results):
        if r['label'].lower() == 'negative':
            combined_list.append((p, n, r['score']))
    sorted_combined = sorted(combined_list, key=lambda x: x[2], reverse=True)
    negative_passages = [(p, n) for p, n, score in sorted_combined]
    return negative_passages[:top_k]


def sort_passages(query_list, passage_list):
    passage_scores = []
    for passage_text, passage_id in passage_list:
        score = get_scores_nbbert(query_list, passage_text)

        if get_tokens_num(passage_text) > 30:
          passage_scores.append((score, passage_text, passage_id))


    sorted_passages = sorted(passage_scores, key=lambda x: x[0], reverse=False)
    return sorted_passages


def opinion_attack(passage_list, query_list, label):
    attack_passages = []

    for i in range(len(passage_list)):
        if len(attack_passages) == 5:
            break

        tar_passage = passage_list[i][1]
        num = passage_list[i][2]
        ori_score = get_scores_nbbert(query_list, tar_passage)

        pas_list, _ = final_attack(query_list, tar_passage, label, 3, 5, 10, 3)
        pas = find_best_passage(pas_list, tar_passage, query_list)
        score = get_scores_nbbert(query_list, pas)
        boost_score = score - ori_score
        pas_input = pas
        attack_passages.append((pas_input, num))

    final_dict = [{"num": attack[1], "attack_passage": attack[0]} for attack in attack_passages]
    return final_dict

config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cuda


In [ ]:
path='/content/opinion_data.json'

import json

def knowledge_guided_attack(t,label):#t是topic在PROCON_data.json中对应的顺序index
  with open(path, "r") as json_file:
      data = json.load(json_file)

  example=data[t]
  category=example['category']
  if category== 'Society & Culture':

    query_list=example['queries']
    query_list.append(example['topic'])
    print(example['topic'])
    passage_list_0=[]
    passage_list_1=[]
    for item in example['passages']:
      if item[1]==0:
        passage_list_0.append((item[3],item[0]))
      elif item[1]==1:
        passage_list_1.append((item[3],item[0]))
    print(len(example['passages']))
    print(len(passage_list_1))
    print(len(passage_list_0))
    if label==0:
      passage_list_0=sort_passages(query_list,passage_list_0)
      know_dict_0=opinion_attack(passage_list_0,query_list,0)
      with open(f"opinion_know_attack_data_{t}_0_token_limit_0.3.json", "w") as json_file:
          json.dump(know_dict_0, json_file, indent=4)
    elif label == 1:
      passage_list_1=sort_passages(query_list,passage_list_1)
      know_dict_1=opinion_attack(passage_list_1,query_list,1)
      with open(f"know_attack_data_{t}_0_k1.json", "w") as json_file:
        json.dump(know_dict_1, json_file, indent=4)

#### know-attack generation action!

In [ ]:
for i in range(20,30):
  attack_t(i,0)

Should Historic Statues Be Taken Down?
44
22
22
Average Score: -0.7703726326043789
Average Score: -2.9721739429693956
Average Score: -0.008122030072487317
Average Score: -3.020267667678686
Average Score: -2.969398872210429
Average Score: 1.171475459749882
Average Score: -2.2686663820193362
Average Score: -4.455168742399949
Average Score: -4.459297253535344
Average Score: -4.444569385968721
Average Score: -4.496949250881489
Average Score: -4.346181897016672
Average Score: -3.974307417869568
Average Score: -4.4689271083244915
Average Score: -2.996947462742145
Average Score: -1.928381052154761
Average Score: -3.5287140149336595
Average Score: -0.8584726971502488
Average Score: -1.9892360881830637
Average Score: -3.8148240676293006
Average Score: -3.6644336260282078
Average Score: -2.049690168637496
Average Score: -4.496949250881489
Grant came from an abolitionist family, freed the one slave he owned in 1859, supported Black enlisted Army men, led the Union Army in the Civil War to abolish